# Label speaker by click buttons

By LongGang Pang

Try to use the widgets in jupyter notebook to design one software to label the data using clicking on 
buttons.

In [1]:
from sanguo import talks

In [2]:
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
from IPython.display import clear_output
import json
import ast
# displaying the labelling widget
##text = widgets.Text(description="Label the speaker by clicking buttons", width=200)
##display(text)

/Users/zhangchenhan/anaconda3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
def check_existence(sentence, idx, saved_file="label_sanguo.txt"):
    speakers = []
    contexts = []
    combined_res = [] 
    # combine N speakers with M contexts to get N*M examples
    with open(saved_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            #res = json.loads(line)
            res = ast.literal_eval(line)
            speakers.append(res['speaker'])
            ctx = res['context']
            ctx_left = ctx[:res["istart"]]
            ctx_right = ctx[res["iend"]:]
            contexts.append([ctx_left, ctx_right, res["istart"]])
            
    for speaker in speakers:
        for ctx in contexts:
            try:
                new_ctx = ctx[0] + speaker + ctx[1]
                istart = ctx[2]
                new_iend = istart + len(speaker)
                res = {'uid':0, 
                   'context':new_ctx,
                   'speaker':speaker,
                   'istart':istart, 
                   'iend':new_iend}
                combined_res.append(res)
                #print(res)
                if sentence == res['context']:
                    print("This item exist:", sentence)
                    res['uid'] = idx
                    return True, res
            except:
                continue
    #print(combined_res)
    return False, None

#check_existence("fdsa", 0)

In [4]:
class ToButtons(object):
    def __init__(self, input_str):
        self.input_str = input_str
        self.res = None
        self.buttons = []
        self.pos_labels = []
        item_layout = Layout(height='40px', min_width='40px', max_width='40px')
        items = [Button(layout=item_layout, 
                        description=input_str[i],
                        tooltip = str(i),
                        button_style='info')
                 for i in range(len(input_str))]
        print(input_str)
        
        box_layout = Layout(
                    border='3px solid black',
                    width='800px',
                    height='',
                    flex_flow='row wrap',
                    display='flex')
        carousel = Box(children=items, layout=box_layout)
        display(carousel)
        
        for item in items[:-2]:
            item.on_click(self.on_button_clicked)
        
            
    # function to deal with the checkbox update button       
    def on_button_clicked(self, b):
        self.pos_labels.append(int(b.tooltip))
        if len(self.pos_labels) > 2:
            #raise ValueError("only click the start and the end word")
            print("Warning: click more than 2 times, will use the last click \
                  as the end of the label position")
            
    def return_results(self):
        if len(self.pos_labels) == 0:
            self.pos_labels.append(-1)
        return self.input_str, self.pos_labels[0], self.pos_labels[-1]+1


    
class LabelSpeaker(object):
    def __init__(self, talk_list, progress=0, save_to='label_sanguo.txt'):
        self.progress = progress
        self.saving_path = save_to
        self.talk_list = talk_list
        self.sentence_buttons = ToButtons(talk_list[self.progress]['context'])
        self.submit_layout = Layout(height='40px', min_width='40px')
        self.submit = Button(layout=self.submit_layout, 
                        description="submit",
                       button_style='warning')
        self.submit.on_click(self.on_button_submit)
        display(self.submit)        

        
    def save_one_item(self, progress, sentence, istart, iend):
        speaker = None
        if istart != -1: speaker = sentence[istart:iend]
        res = {'uid':progress, 'context':sentence,
               'speaker':speaker,
               'istart':istart, 'iend':iend}
        with open(self.saving_path, 'a') as f:
            f.write(res.__repr__())
            f.write('\n')
        
    def on_button_submit(self, b):
        sentence, istart, iend = self.sentence_buttons.return_results()
        print(sentence[istart:iend])
        self.save_one_item(self.progress, sentence, istart, iend)
        clear_output()
        while True:
            self.progress = self.progress + 1
            new_sentence = self.talk_list[self.progress]['context']
            existed, res = check_existence(new_sentence, self.progress)
            if existed: 
                self.save_one_item(self.progress, new_sentence,
                                      res['istart'], res['iend'])
            else:
                break
        #### after check exist 
        self.sentence_buttons = ToButtons(new_sentence)
        self.submit = Button(layout=self.submit_layout, 
                        description="submit",
                       button_style='warning')
        self.submit.on_click(self.on_button_submit)
        display(self.submit)

### 

In [5]:
b1 = LabelSpeaker(talks, progress=3263)

When the troop of advisers had retired, Lu Su came in, saying, “


Box(children=(Button(button_style='info', description='W', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='0'), Button(button_style='info', description='h', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='1'), Button(button_style='info', description='e', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='2'), Button(button_style='info', description='n', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='3'), Button(button_style='info', description=' ', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='4'), Button(button_style='info', description='t', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='5'), Button(button_style='info', description='h', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='6'), Button(button_style='info', description='e', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='7'), Button(button_style='info', description=' ', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='8'), Button(button_style='info', description='t', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='9'), Button(button_style='info', description='r', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='10'), Button(button_style='info', description='o', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='11'), Button(button_style='info', description='o', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='12'), Button(button_style='info', description='p', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='13'), Button(button_style='info', description=' ', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='14'), Button(button_style='info', description='o', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='15'), Button(button_style='info', description='f', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='16'), Button(button_style='info', description=' ', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='17'), Button(button_style='info', description='a', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='18'), Button(button_style='info', description='d', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='19'), Button(button_style='info', description='v', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='20'), Button(button_style='info', description='i', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='21'), Button(button_style='info', description='s', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='22'), Button(button_style='info', description='e', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='23'), Button(button_style='info', description='r', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='24'), Button(button_style='info', description='s', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='25'), Button(button_style='info', description=' ', layout=Layout(height='40px', max_width='40px', min_width='40px'), style=ButtonStyle(), tooltip='26'), Button(button_style='info',

Button(button_style='warning', description='submit', layout=Layout(height='40px', min_width='40px'), style=ButtonStyle())